In [12]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
#import sweetviz as sv
import HelperFunctions as hf


# Overview

[M5 forecasting challenge](https://www.kaggle.com/c/m5-forecasting-accuracy/data)


## Data Description

- `calendar.csv` - Contains information about the dates on which the products are sold.

- `sales_train_validation.csv` - Contains the historical daily unit sales data per product and store `[d_1 - d_1913]`

- `sell_prices.csv` - Contains information about the price of the products sold per store and date.

- `sample_submission.csv` - The correct format for submissions. Reference the [Evaluation](https://www.kaggle.com/c/m5-forecasting-accuracy/overview/evaluation) tab for more info.

- `sales_train_evaluation.csv` - Includes sales `[d_1 - d_1941]` (labels used for the Public leaderboard)


## Task

- Forecast daily sales for the next 28 days



# 1. Download the Data

In [5]:
%%bash

kaggle competitions download m5-forecasting-accuracy -p data


100%|██████████| 45.8M/45.8M [00:12<00:00, 3.83MB/s]


In [6]:
import os 
import tarfile
import zipfile

def fetch_walmat_data():
    zip_path = os.path.join('data', 'm5-forecasting-accuracy.zip')    
    
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall('data')
    
    print(zip_ref.namelist())

fetch_walmat_data()


['calendar.csv', 'sales_train_evaluation.csv', 'sales_train_validation.csv', 'sample_submission.csv', 'sell_prices.csv']


# 2. Read the Data

In [7]:
def load_calendar_data():    
    csv_path = os.path.join('data', 'calendar.csv')
    return pd.read_csv(csv_path)


def load_sell_price_data():
    csv_path = os.path.join('data', 'sell_prices.csv')
    return pd.read_csv(csv_path)


def load_sales_train_validation_data():
    csv_path = os.path.join('data', 'sales_train_validation.csv')
    return pd.read_csv(csv_path)


def load_sales_train_evaluation_data():
    csv_path = os.path.join('data', 'sales_train_evaluation.csv')
    return pd.read_csv(csv_path)


## 2.1 Read Calendar

In [8]:
calendar = load_calendar_data()
calendar.head()


,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


In [9]:
print(f'shape: {calendar.shape}\n')

calendar.info()

shape: (1969, 14)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1969 entries, 0 to 1968
Data columns (total 14 columns):
date            1969 non-null object
wm_yr_wk        1969 non-null int64
weekday         1969 non-null object
wday            1969 non-null int64
month           1969 non-null int64
year            1969 non-null int64
d               1969 non-null object
event_name_1    162 non-null object
event_type_1    162 non-null object
event_name_2    5 non-null object
event_type_2    5 non-null object
snap_CA         1969 non-null int64
snap_TX         1969 non-null int64
snap_WI         1969 non-null int64
dtypes: int64(7), object(7)
memory usage: 215.5+ KB


### Sweetviz

In [22]:
# quick eda by sweetViz
calendar_report = sv.analyze(calendar)

# display the report
calendar_report.show_html('Calendar.html')



Report Calendar.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [10]:
#Read data files
price_df = load_sell_price_data()
sales_df = load_sales_train_evaluation_data()

In [26]:
#Create the melted dataframe
df = hf.meltM5(sales_df, days = 800, items = 10000)

In [27]:
#Join with calendar and price data
df = hf.joinDataSets(df, calendar, price_df, dropPriceNA=False)

Initial inspection of the raw data

In [28]:
df.shape

(8000000, 22)

In [29]:
df.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sold,date,wm_yr_wk,...,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOUSEHOLD_1_032_CA_2_evaluation,HOUSEHOLD_1_032,HOUSEHOLD_1,HOUSEHOLD,CA_2,CA,d_1,0,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
1,HOBBIES_2_123_TX_1_evaluation,HOBBIES_2_123,HOBBIES_2,HOBBIES,TX_1,TX,d_1,0,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
2,HOBBIES_2_099_CA_4_evaluation,HOBBIES_2_099,HOBBIES_2,HOBBIES,CA_4,CA,d_1,0,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
3,HOUSEHOLD_1_493_TX_3_evaluation,HOUSEHOLD_1_493,HOUSEHOLD_1,HOUSEHOLD,TX_3,TX,d_1,0,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
4,FOODS_2_359_CA_1_evaluation,FOODS_2_359,FOODS_2,FOODS,CA_1,CA,d_1,2,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,2.23


In [30]:
df.describe()

,sold,wm_yr_wk,wday,month,year,snap_CA,snap_TX,snap_WI,sell_price
count,8.000000e+06,8.000000e+06,8.000000e+06,8.000000e+06,8.000000e+06,8.000000e+06,8.000000e+06,8.000000e+06,4.741225e+06
mean,1.009291e+00,1.118812e+04,3.993750e+00,6.182500e+00,2.011700e+03,3.337500e-01,3.312500e-01,3.300000e-01,4.320895e+00
std,4.302489e+00,6.202339e+01,2.001552e+00,3.481982e+00,6.726812e-01,4.715517e-01,4.706628e-01,4.702127e-01,3.195661e+00
min,0.000000e+00,1.110100e+04,1.000000e+00,1.000000e+00,2.011000e+03,0.000000e+00,0.000000e+00,0.000000e+00,5.000000e-02
25%,0.000000e+00,1.112900e+04,2.000000e+00,3.000000e+00,2.011000e+03,0.000000e+00,0.000000e+00,0.000000e+00,2.080000e+00
50%,0.000000e+00,1.120600e+04,4.000000e+00,6.000000e+00,2.012000e+03,0.000000e+00,0.000000e+00,0.000000e+00,3.340000e+00
75%,1.000000e+00,1.123400e+04,6.000000e+00,9.000000e+00,2.012000e+03,1.000000e+00,1.000000e+00,1.000000e+00,5.840000e+00
max,6.930000e+02,1.131100e+04,7.000000e+00,1.200000e+01,2.013000e+03,1.000000e+00,1.000000e+00,1.000000e+00,3.098000e+01


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8000000 entries, 0 to 7999999
Data columns (total 22 columns):
id              object
item_id         object
dept_id         object
cat_id          object
store_id        object
state_id        object
d               object
sold            int64
date            object
wm_yr_wk        int64
weekday         object
wday            int64
month           int64
year            int64
event_name_1    object
event_type_1    object
event_name_2    object
event_type_2    object
snap_CA         int64
snap_TX         int64
snap_WI         int64
sell_price      float64
dtypes: float64(1), int64(8), object(13)
memory usage: 1.4+ GB


In [37]:
#These are likely seasonal products, such as xmas trees, or brussel sprouts
print("Percentage of products that are not on sale: ", df['sell_price'].isna().sum() / len(df['sell_price']) * 100)

Percentage of products that are not on sale:  40.7346875
